In [1]:
import pandas as pd
import json
from csv import reader, writer
import os
from copy import deepcopy

In [2]:
dir="../benchmark/sap"
# file="benchmark_v5_fixed_filtered.csv"
file="v5_test_benchmark.csv"
out_file="v5_test_benchmark.jsonl"
loio_map="en_loio_map.tsv"
use_loios = True
loio2id = {}
build_loio = True
corpus="extended_components.jsonl"
docdata = []

In [12]:
from tqdm import tqdm


def extract_loio(d):
    return d['url'].split("/")[-1].replace(".html","")

if build_loio:
    loio2id = {}
    with open(os.path.join(dir,corpus),"r") as f:
        for line in tqdm(f):
            docdata = json.loads(line)
            loio = extract_loio(docdata)
            if loio not in loio2id:
                loio2id[loio] = [docdata['document_id']]
            else:
                loio2id[loio].append(docdata['document_id'])

else:
    with open(os.path.join(dir, loio_map),"r") as f:
        r1 = reader(f, delimiter="\t")
        for row in tqdm(r1):
            if row[1] not in loio2id:
                loio2id[row[1]] = [row[0]]
            else:
                loio2id[row[1]].append(row[0])


73511it [00:15, 4655.21it/s]


In [13]:
[k for k,v in loio2id.items() if len(v)>1]

['bade65d5ffd049249bbd22cc0c672898',
 '3ca24f739bc740d9938a6fed77c05b5f',
 'b1d0a7e300b244dda5f77890ba66ba82',
 '1e3d7bed916b414a969aa690b89afe0c',
 '51f37e43ceab4444b752111f7868a482',
 'b51e1029eae74fa4b68e78b10e8a5f50',
 '70650db363264eb39e16bd089de97138',
 '8aeb6c48ea1b451aa27e34f1d025191e',
 'ab71572cd4064fa3b3641e7eb22167d2',
 '37feb753128eb44ce10000000a174cb4',
 '52feb753128eb44ce10000000a174cb4',
 'fc8cbc7c82814462a9a6d67867dcddf6',
 '8bcac5ee24d1484dada9c6fad34ba257',
 '76fcb4e7ecd744e298643f7d2bc617ad',
 'cb2b3307a05c4d669d04d2bf13c26b38',
 'd4ca330216f0418b87d9418c87b2b569',
 'b88a07aa0f144bcdbcf14d130bd314c2',
 'ab8ae860631c47fca1a2e0913f63f610',
 'f90ca257107aa30be10000000a44147b',
 '0443bfdffb2a47f2a3408e5db2ff179b',
 '0cd6feecdb084c8f9a090c295a90d2a7',
 'a8c390b7e0b14ebcb82753ca78ad7d69',
 '97e9ee571a3e1470e10000000a44147b',
 '1c5ef1f414f74b1bba6967346c59c1d6',
 '66ffc5536a51204be10000000a174cb4',
 '7fea9ae4cf3b4fae84e83e0916b87324',
 '13551d6ccbb8433f9c8362b354534bed',
 

In [14]:
loio2id['9f301385d30f4c32924590548c85f164']

['sap_38437.txt']

In [15]:
r=reader(open(os.path.join(dir, file), "r"), delimiter=",")
data=[l for l in r]

In [32]:
res=[]
seen = {}
for d in data:
    d1 = deepcopy(d)
    if d1[0] not in seen:
        seen[d1[0]]=True
        o = []
        if d1[0]=='Count':
            d1.append("relevant")
        else:
            for i in range(6,9):
                if d1[i]!='':
                    o.append(d1[i].replace("loio", ""))
            # print(d1)
            if use_loios:
                o1 = []
                for loio in o:
                    try:
                        o1.extend(loio2id[loio])
                    except KeyError as e:
                        print(f"Key {e} not found")
                o = o1
            d1.append(o)
        res.append(d1)

In [35]:
res[3]

['2',
 '8',
 'Indirect',
 'Here\'s a summary of what I have found for you: \\n \\n Yes, you can update your login account information. As an administrator, you have the ability to manage login accounts. To update your login account details, follow these steps:\\n\\n1. Go to Admin Center.\\n2. Navigate to Manage Login Accounts.\\n3. Search for your login account using your first name, last name, username, or person ID.\\n4. Select your account and choose "Edit Details".\\n5. Update the necessary information such as login name, login method, language, and time zone.\\n6. If needed, choose to send a notification to the account user.\\n7. Save your changes.\\n\\nBy following these steps, you will be able to update your login account information successfully.',
 'SAP_SUCCESSFACTORS',
 'My login account info is not correct. Can I update it?',
 'loioc99a89da1eba4e8b942cd7280b1b8a44',
 '',
 '',
 ['sap_168727.txt']]

In [36]:
with open(os.path.join(dir, out_file), "w") as out:
    headers = res[0]
    for d in res[1:]:
        out.write(json.dumps({headers[i]:d[i] for i in range(0,len(res[0]))})+"\n")

In [20]:
len(res)

244

In [26]:
headers[9]

'relevant'